In [3]:
import os
import jieba
from sklearn.naive_bayes import MultinomialNB  # 基于多项式模型的朴素贝叶斯
from sklearn.model_selection import train_test_split

# 以下代码可通过调用`from sklearn.feature_extraction.text import CountVectorizer`简洁实现

In [4]:
def text_processing(folder_path, test_size=0.2):
    """
    文本生成过程
    :param folder_path:目录路径
    :param test_size:测试集数据比例
    :return:不重复的单词,训练集样本,测试集样本,训练集标签,测试集标签
    """
    folder_list = os.listdir(folder_path)  # folder_path下的所有目录,返回值为列表
    data_list = []  # 每个文件的分词
    class_list = []  # 每个文件的标签

    for folder in folder_list:
        new_folder_path = os.path.join(folder_path, folder)  # 路径拼接
        files = os.listdir(new_folder_path)  # new_folder_path下的所有.txt文件

        for file in files:
            with open(os.path.join(new_folder_path, file), 'r', encoding='UTF-8', errors='ignore') as fp:
                raw = fp.read()
            word_cut = jieba.cut(raw, cut_all=False)  # 精确模式,返回值为生成器
            word_list = list(word_cut)
            data_list.append(word_list)
            class_list.append(folder)

    train_data_list, test_data_list, train_class_list, test_class_list = train_test_split(data_list, class_list,
                                                                                          test_size=test_size)  # 数据集划分

    all_words_dict = {}  # 词频统计
    for word_list in train_data_list:
        for word in word_list:
            if word in all_words_dict:
                all_words_dict[word] += 1
            else:
                all_words_dict[word] = 1

    all_words_tuple_list = sorted(all_words_dict.items(), key=lambda f: f[1], reverse=True)  # 根据词频降序排列
    all_words_list, _ = zip(*all_words_tuple_list)  # 解包
    all_words_list = list(all_words_list)

    return all_words_list, train_data_list, test_data_list, train_class_list, test_class_list

In [6]:
# text_processing函数测试
folder_path = 'Sample'  # 包含9中不同新闻的总文件夹
all_words_list, train_data_list, test_data_list, train_class_list, test_class_list = text_processing(folder_path,
                                                                                                     test_size=0.2)
print(len(all_words_list))  # 不重复单词数
print(all_words_list[:20])

print(len(train_data_list))  # 72个训练集样本
print(len(test_data_list))  # 18个测试集样本

print(len(train_class_list))  # 72个训练集标签
print(len(test_class_list))  # 18个测试集标签

10391
['，', '的', '\u3000', '。', '\n', ';', '&', ' ', 'nbsp', '在', '、', '了', '“', '”', '是', '和', '：', '\x00', '我', '中国']
72
18
72
18


In [7]:
def make_word_set(words_file):
    """生成停用词(去重)"""
    words_set = set()  # 集合
    with open(words_file, 'r', encoding='UTF-8', errors='ignore') as fp:
        for line in fp.readlines():  # 循环取出每一行
            word = line.strip()
            if len(word) > 0 and word not in words_set:  # 去重
                words_set.add(word)
    return words_set

In [8]:
stopwords_file = './stopwords_cn.txt'  # 停用词文件
stopwords_set = make_word_set(stopwords_file)  # 首先停用词去重
print(len(stopwords_set))
print(stopwords_set)  # 所有停用词

428
{'诸', '唯有', '把', '哪个', '不至于', '别说', '格里斯', '向', '对于', '哇', '看', '只怕', '因之', '这儿', '再有', '随时', '据', '以来', '反之', '些', '多么', '就算', '一', '当然', '各', '并不', '此', '与', '靠', '本着', '为着', '对待', '什么样', '根据', '小', '各位', '许多', '同时', '就是', '朝着', '前者', '要不', '直到', '其余', '不但', '至今', '及', '由', '她们', '跟', '因', '此间', '任何', '之所以', '则', '况且', '另外', '不如', '以便', '出来', '我们', '只消', '等等', '虽然', '再则', '正如', '何况', '及至', '你', '受到', '且', '最', '后者', '而且', '替代', '很', '被', '去', '或者说', '又', '以为', '如是', '无', '如果', '咱', '处在', '着', '如若', '依据', '继而', '然而', '一切', '为止', '吧', '分别', '诸位', '可是', '某个', '多会', '他', '总之', '尽管如此', '那么', '沿着', '针对', '介于', '打', '这么', '因此', '比', '说来', '与否', '可', '用来', '而已', '否则', '才能', '凡是', '至', '其', '如', '例如', '为何', '并非', '不过', '其中', '她', '其他', '还要', '然后', '或者', '不外乎', '甚至于', '某某', '哪些', '既然', '不管', '来说', '只', '还是', '了', '而', '趁', '那边', '拿', '此时', '对比', '怎么办', '甚至', '他人', '随', '大家', '万一', '个', '什么', '由此', '譬如', '不', '为什么', '但是', '仍旧', '什么的', '它', '只要', '于', '各自', '不单', '本地', '某些', '后', '自己', '要不然'

In [9]:
def words_dict(all_words_list, deleteN, stopwords_set=None):
    """特征词选取"""
    if stopwords_set is None:
        stopwords_set = set()
    feature_words = []
    n = 1
    for t in range(deleteN, len(all_words_list), 1):
        if n > 1000:  # feature_words的维度这里选定频率前1000的词语
            break

        if not all_words_list[t].isdigit() and all_words_list[t] not in stopwords_set and 1 < len(
                all_words_list[t]) < 5:  # 这里设定特征词需满足三个条件:不是数字;不在停用词表;长度2~4
            feature_words.append(all_words_list[t])
        n += 1

    return feature_words

In [10]:
deleteN = 20  # 舍弃频率前20的词
# 越靠前的词语出现的越频繁,有可能所有类别中都出现很多次,这类词去掉效果可能更好
feature_words = words_dict(all_words_list, deleteN, stopwords_set)
print(feature_words[:100])

['一个', '游客', '公司', '导弹', '大陆', '旅游', '北京', '市场', '火炮', '认为', '进行', '台军', '考生', '没有', '已经', '时间', '各种', '志愿', '一种', '美国', '解放军', '作战', '成为', '企业', '支付', '仿制', '主要', '目前', '比赛', '问题', '发展', '很多', '远程', '可能', '五一', '黄金周', '通过', '射程', '选择', '完全', '分析', '记者', '技术', '学校', '词汇', '部署', '能力', '增长', '时候', '表示', '建设', '文章', '需要', '专业', '学习', '工作', '一定', '今年', '期间', '接待', '万人次', '毕业生', '开始', '部分', '填报', '亿美元', '使用', '部队', '情况', '现在', '专家', '电话', '比较', '相对', '阵地', '提供', '科学', '拥有', '资料', '必须', '达到', '收入', '销售', '几乎', '军事', '表现', '用户', '考试', '计划', '影响', '这是', '历史', '人数', '服务', '训练', '产品', '来源', '希望', '复习', '最后']


In [11]:
def text_features(train_data_list, test_data_list, feature_words):
    """用0/1判断是否出现在特征词中"""

    def text_features(text, feature_words):  # text的定义在下面
        text_words = set(text)  # 样本去重
        # 遍历每个样本词语,凡是样本的词语出现在1000个特征词里,就记录下来为1,否则为0
        features = [1 if word in text_words else 0 for word in feature_words]
        return features

    train_feature_list = [text_features(text, feature_words) for text in train_data_list]
    test_feature_list = [text_features(text, feature_words) for text in test_data_list]

    return train_feature_list, test_feature_list

In [12]:
train_feature_list, test_feature_list = text_features(train_data_list, test_data_list, feature_words)
print(len(train_feature_list))
print(len(test_feature_list))
print(len(test_feature_list[0]))  # 每个样本的维度都是1000
print(test_feature_list[4][:200])  # 打印测试集的第5个样本的前200个值

72
18
1000
[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]


In [13]:
def text_classifier(train_feature_list, test_feature_list,
                    train_class_list, test_class_list):
    """朴素贝叶斯分类,同时输出准确率"""
    classifier = MultinomialNB().fit(train_feature_list, train_class_list)
    test_accuracy = classifier.score(test_feature_list, test_class_list)

    return test_accuracy

In [14]:
test_accuracy = text_classifier(train_feature_list, test_feature_list,
                                train_class_list, test_class_list)
print(test_accuracy)

0.7777777777777778
